In [3]:
import numpy as np
import pandas as pd
import mne
from mne.io import RawArray

from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs
from mne.decoding import CSP
from mne.channels import read_custom_montage
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score, LeaveOneLabelOut
from glob import glob
from sklearn.model_selection import ShuffleSplit, cross_val_score
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.signal import welch
from mne import pick_types

def creat_mne_raw_object(fname):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = 'standard_1005'

    # events file
    ev_fname = fname.replace('_data','_events')
    # read event file
    events = pd.read_csv(ev_fname)
    events_names = events.columns[1:]
    events_data = np.array(events[events_names]).T
    
    # concatenate event file and data
    data = np.concatenate((1e-6*np.array(data[ch_names]).T,events_data))        
    
    # define channel type, the first is EEG, the last 6 are stimulations
    ch_type = ['eeg']*len(ch_names) + ['stim']*6
    
    # create and populate MNE info structure
    ch_names.extend(events_names)
    info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage=montage)
    #info.set_montage(montage)
    #info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    return raw


In [9]:
subjects = range(1,13)
auc = []
for subject in subjects:
    epochs_tot = []
    
    #eid = 'HandStart'
    fnames =  glob('../EEG/train/subj%d_series*_data.csv' % (subject))
    
    session = []
    y = []
    for i,fname in enumerate(fnames):
      
        # read data 
        raw = creat_mne_raw_object(fname)
        #raw.plot(block=True)
        # pick eeg signal
        picks = pick_types(raw.info,eeg=True)
        
        # Filter data for alpha frequency and beta band
        # Note that MNE implement a zero phase (filtfilt) filtering not compatible
        # with the rule of future data.
        raw.filter(7,35, picks=picks, method='iir', n_jobs=-1, verbose=False)
        
        # get event posision corresponding to HandStart
        events = find_events(raw,stim_channel='HandStart', verbose=False)
        # epochs signal for 1.5 second before the movement
        epochs = Epochs(raw, events, {'during' : 1}, 0.5, 2, proj=False, picks=picks, baseline=None, preload=True, verbose=False)
        
        epochs_tot.append(epochs)
        session.extend([i]*len(epochs))
        y.extend([1]*len(epochs))
        
        # epochs signal for 1.5 second after the movement, this correspond to the 
        # rest period.
        epochs_rest = Epochs(raw, events, {'before' : 1}, -2, -0.5, proj=False, picks=picks, baseline=None, preload=True, verbose=False)
        
        # Workaround to be able to concatenate epochs
        epochs_rest.shift_time(2.5, relative = True)
        
        epochs_tot.append(epochs_rest)
        session.extend([i]*len(epochs_rest))
        y.extend([-1]*len(epochs_rest))
        
    #concatenate all epochs
    epochs = concatenate_epochs(epochs_tot)
    # epochs.plot(picks=picks, scalings='auto', show=True, block=True, event_id='during')
    
    # get data 
    X = epochs.get_data()
    y = np.array(y)
    
    #print(X.shape)
    #print(y.shape)
    #print(y)
    #input()
    
    # run CSP
    csp = CSP(reg='ledoit_wolf') #0.20 changed from 'lws' to 'ledoit_wolf'
    csp.fit(X,y)
    csp.fit_transform(X, y)
    
    # set cross_validation (why before 'fit_transform'?)
    clf = make_pipeline(CSP(),LogisticRegression()) 
    #cv = ShuffleSplit(10, test_size=0.2, random_state=42)
    #cv_split = cv.split(X)
    #cv = LeaveOneLabelOut(session)
    
    
    
    # csp.plot_patterns(epochs.info, components = 2, ch_type='eeg', units='Patterns (AU)', size=1.5)
    
    # print cross validation sorces
    auc.append(cross_val_score(clf,X,y,scoring='roc_auc').mean())
    print("Subject %d : AUC cross val score : %.3f" % (subject,auc[-1]))

DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'A1', 'PO1h', 'AFp10', 'TP8h', 'FTT9', 'FC4', 'AFF8h', 'POO5h', 'AFp3', 'C6', 'AFF4h', 'FCC6', 'T7h', 'CPP2', 'AF4', 'AFF1h', 'FFT10h', 'PPO6h', 'AF5', 'I1h', 'FT8', 'FFC1', 'AFpz', 'POO7h', 'AF10', 'P5', 'CP5h', 'PPO9', 'CCP1', 'FT9', 'POO2h', 'C6h', 'AF2', 'T5', 'C1h', 'AFF3', 'FT10', 'AFp6', 'AFF10h', 'OI1h', 'T10h', 'FTT10h', 'AF7h', 'PPO6', 'PPO10h', 'TPP8', 'AFp1', 'PO7', 'TP7', 'PO9h', 'FTT9h', 'FFC3', 'P9', 'FFT9h', 'C1', 'FCC2h', 'AFF6', 'AFp3h', 'AFF7', 'AF10h', 'AFF1', 'AFFz', 'FC1h', 'FFT10', 'FFC6h', 'CP4h', 'C2', 'AFF9h', 'P6h', 'P8h', 'POOz', 'CPP1h', 'FT9h', 'AF1', 'PO7h', 'POz', 'CPP4', 'POO4', 'TPP10h', 'AFF6h', 'P2h', 'CPP3', 'CCP3h', 'AFp9', 'POO2', 'FCC3h', 'PPO8h', 'O1h', 'AFF7h', 'T6', 'FFC2', 'AFz', 'F5h', 'CPP4h', 'FC6h', 'FT10h', 'CPPz', 'TTP7h', 'AFp10h', 'C5', 'FCC1h', 'TPP10', 'F1', 'AF7', 'FC2h', 'CP2h', 'POO6h', 'AFp4h', 'F10h', 'POO8', 'PPOz', 'Fpz', 'AFp7', '

<ipython-input-3-619daede538f>:48: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage=montage)
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/

DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'A1', 'PO1h', 'AFp10', 'TP8h', 'FTT9', 'FC4', 'AFF8h', 'POO5h', 'AFp3', 'C6', 'AFF4h', 'FCC6', 'T7h', 'CPP2', 'AF4', 'AFF1h', 'FFT10h', 'PPO6h', 'AF5', 'I1h', 'FT8', 'FFC1', 'AFpz', 'POO7h', 'AF10', 'P5', 'CP5h', 'PPO9', 'CCP1', 'FT9', 'POO2h', 'C6h', 'AF2', 'T5', 'C1h', 'AFF3', 'FT10', 'AFp6', 'AFF10h', 'OI1h', 'T10h', 'FTT10h', 'AF7h', 'PPO6', 'PPO10h', 'TPP8', 'AFp1', 'PO7', 'TP7', 'PO9h', 'FTT9h', 'FFC3', 'P9', 'FFT9h', 'C1', 'FCC2h', 'AFF6', 'AFp3h', 'AFF7', 'AF10h', 'AFF1', 'AFFz', 'FC1h', 'FFT10', 'FFC6h', 'CP4h', 'C2', 'AFF9h', 'P6h', 'P8h', 'POOz', 'CPP1h', 'FT9h', 'AF1', 'PO7h', 'POz', 'CPP4', 'POO4', 'TPP10h', 'AFF6h', 'P2h', 'CPP3', 'CCP3h', 'AFp9', 'POO2', 'FCC3h', 'PPO8h', 'O1h', 'AFF7h', 'T6', 'FFC2', 'AFz', 'F5h', 'CPP4h', 'FC6h', 'FT10h', 'CPPz', 'TTP7h', 'AFp10h', 'C5', 'FCC1h', 'TPP10', 'F1', 'AF7', 'FC2h', 'CP2h', 'POO6h', 'AFp4h', 'F10h', 'POO8', 'PPOz', 'Fpz', 'AFp7', '

<ipython-input-3-619daede538f>:48: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage=montage)
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/

DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'A1', 'PO1h', 'AFp10', 'TP8h', 'FTT9', 'FC4', 'AFF8h', 'POO5h', 'AFp3', 'C6', 'AFF4h', 'FCC6', 'T7h', 'CPP2', 'AF4', 'AFF1h', 'FFT10h', 'PPO6h', 'AF5', 'I1h', 'FT8', 'FFC1', 'AFpz', 'POO7h', 'AF10', 'P5', 'CP5h', 'PPO9', 'CCP1', 'FT9', 'POO2h', 'C6h', 'AF2', 'T5', 'C1h', 'AFF3', 'FT10', 'AFp6', 'AFF10h', 'OI1h', 'T10h', 'FTT10h', 'AF7h', 'PPO6', 'PPO10h', 'TPP8', 'AFp1', 'PO7', 'TP7', 'PO9h', 'FTT9h', 'FFC3', 'P9', 'FFT9h', 'C1', 'FCC2h', 'AFF6', 'AFp3h', 'AFF7', 'AF10h', 'AFF1', 'AFFz', 'FC1h', 'FFT10', 'FFC6h', 'CP4h', 'C2', 'AFF9h', 'P6h', 'P8h', 'POOz', 'CPP1h', 'FT9h', 'AF1', 'PO7h', 'POz', 'CPP4', 'POO4', 'TPP10h', 'AFF6h', 'P2h', 'CPP3', 'CCP3h', 'AFp9', 'POO2', 'FCC3h', 'PPO8h', 'O1h', 'AFF7h', 'T6', 'FFC2', 'AFz', 'F5h', 'CPP4h', 'FC6h', 'FT10h', 'CPPz', 'TTP7h', 'AFp10h', 'C5', 'FCC1h', 'TPP10', 'F1', 'AF7', 'FC2h', 'CP2h', 'POO6h', 'AFp4h', 'F10h', 'POO8', 'PPOz', 'Fpz', 'AFp7', '

...

DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'A1', 'PO1h', 'AFp10', 'TP8h', 'FTT9', 'FC4', 'AFF8h', 'POO5h', 'AFp3', 'C6', 'AFF4h', 'FCC6', 'T7h', 'CPP2', 'AF4', 'AFF1h', 'FFT10h', 'PPO6h', 'AF5', 'I1h', 'FT8', 'FFC1', 'AFpz', 'POO7h', 'AF10', 'P5', 'CP5h', 'PPO9', 'CCP1', 'FT9', 'POO2h', 'C6h', 'AF2', 'T5', 'C1h', 'AFF3', 'FT10', 'AFp6', 'AFF10h', 'OI1h', 'T10h', 'FTT10h', 'AF7h', 'PPO6', 'PPO10h', 'TPP8', 'AFp1', 'PO7', 'TP7', 'PO9h', 'FTT9h', 'FFC3', 'P9', 'FFT9h', 'C1', 'FCC2h', 'AFF6', 'AFp3h', 'AFF7', 'AF10h', 'AFF1', 'AFFz', 'FC1h', 'FFT10', 'FFC6h', 'CP4h', 'C2', 'AFF9h', 'P6h', 'P8h', 'POOz', 'CPP1h', 'FT9h', 'AF1', 'PO7h', 'POz', 'CPP4', 'POO4', 'TPP10h', 'AFF6h', 'P2h', 'CPP3', 'CCP3h', 'AFp9', 'POO2', 'FCC3h', 'PPO8h', 'O1h', 'AFF7h', 'T6', 'FFC2', 'AFz', 'F5h', 'CPP4h', 'FC6h', 'FT10h', 'CPPz', 'TTP7h', 'AFp10h', 'C5', 'FCC1h', 'TPP10', 'F1', 'AF7', 'FC2h', 'CP2h', 'POO6h', 'AFp4h', 'F10h', 'POO8', 'PPOz', 'Fpz', 'AFp7', '

<ipython-input-3-619daede538f>:48: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage=montage)
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/

DigMontage is a superset of info. 311 in DigMontage will be ignored. The ignored channels are: {'A1', 'PO1h', 'AFp10', 'TP8h', 'FTT9', 'FC4', 'AFF8h', 'POO5h', 'AFp3', 'C6', 'AFF4h', 'FCC6', 'T7h', 'CPP2', 'AF4', 'AFF1h', 'FFT10h', 'PPO6h', 'AF5', 'I1h', 'FT8', 'FFC1', 'AFpz', 'POO7h', 'AF10', 'P5', 'CP5h', 'PPO9', 'CCP1', 'FT9', 'POO2h', 'C6h', 'AF2', 'T5', 'C1h', 'AFF3', 'FT10', 'AFp6', 'AFF10h', 'OI1h', 'T10h', 'FTT10h', 'AF7h', 'PPO6', 'PPO10h', 'TPP8', 'AFp1', 'PO7', 'TP7', 'PO9h', 'FTT9h', 'FFC3', 'P9', 'FFT9h', 'C1', 'FCC2h', 'AFF6', 'AFp3h', 'AFF7', 'AF10h', 'AFF1', 'AFFz', 'FC1h', 'FFT10', 'FFC6h', 'CP4h', 'C2', 'AFF9h', 'P6h', 'P8h', 'POOz', 'CPP1h', 'FT9h', 'AF1', 'PO7h', 'POz', 'CPP4', 'POO4', 'TPP10h', 'AFF6h', 'P2h', 'CPP3', 'CCP3h', 'AFp9', 'POO2', 'FCC3h', 'PPO8h', 'O1h', 'AFF7h', 'T6', 'FFC2', 'AFz', 'F5h', 'CPP4h', 'FC6h', 'FT10h', 'CPPz', 'TTP7h', 'AFp10h', 'C5', 'FCC1h', 'TPP10', 'F1', 'AF7', 'FC2h', 'CP2h', 'POO6h', 'AFp4h', 'F10h', 'POO8', 'PPOz', 'Fpz', 'AFp7', '

<ipython-input-3-619daede538f>:48: RuntimeWarning: 1 channel names are too long, have been truncated to 15 characters:
['BothStartLoadPhase']
  info = create_info(ch_names,sfreq=500.0,ch_types=ch_type,montage=montage)
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]
/home/yaoxiaojian/anaconda3/

KeyboardInterrupt: 